In [26]:
!pip install --upgrade pip
!pip install fosforml numpy pandas matplotlib scikit-learn seaborn python-dateutil
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 
!pip install fosforio
!pip install refractio
!pip install refractml

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached pip-24.1.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.1.2-py3-none-any.whl (1.8 MB)
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached fosforml-1.1.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached numpy-2.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pandas-2.2.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached matplotlib-3.9.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scikit_learn-1.5.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached seaborn-0.13.2-py3-non

  Using cached pyOpenSSL-24.1.0-py3-none-any.whl.metadata (12 kB)
  Using cached PyJWT-2.8.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached charset_normalizer-3.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached platformdirs-4.2.2-py3-none-any.whl.metadata (11 kB)
  Using cached tomlkit-0.13.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached urllib3-1.26.19-py2.py3-none-any.whl.metadata (49 kB)
  Using cached setuptools-70.3.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached botocore-1.34.131-py3-none-any.whl.metadata (5.7 kB)
  Using cached wrapt-1.16.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Usin

Using cached filelock-3.15.4-py3-none-any.whl (16 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached platformdirs-4.2.2-py3-none-any.whl (18 kB)
Using cached PyJWT-2.8.0-py3-none-any.whl (22 kB)
Using cached pyOpenSSL-24.1.0-py3-none-any.whl (56 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached setuptools-70.3.0-py3-none-any.whl (931 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
Using cached urllib3-1.26.19-py2.py3-none-any.whl (143 kB)
Using cached exceptiongroup-1.2.2-py3-none-any.whl (16 kB)
Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)
Using cached plotly-5.22.0-py3-none-any.whl (16.4 MB)
Using cached tomlkit-0.13.0-py3-none-any.whl (37 kB)
Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
Using cached aioitertools-0.11.0-py3-none-any.whl (23 kB)
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
Using c

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Found existing installation: urllib3 1.26.15
Uninstalling urllib3-1.26.15:
  Successfully uninstalled urllib3-1.26.15
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached urllib3-1.26.15-py2.py3-none-any.whl.metadata (48 kB)
Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached fosforml-1.1.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached scikit_learn-1.3.2-cp39-cp39-manylinux_2_17_x86_64.

  Using cached PyJWT-2.8.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached charset_normalizer-3.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached platformdirs-4.2.2-py3-none-any.whl.metadata (11 kB)
  Using cached tomlkit-0.13.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached urllib3-1.26.19-py2.py3-none-any.whl.metadata (49 kB)
  Using cached setuptools-70.3.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached botocore-1.34.131-py3-none-any.whl.metadata (5.7 kB)
  Using cached wrapt-1.16.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached aioitertools-0.11.0-py3-none-any.whl.metadata (3.3 kB)
  

Using cached filelock-3.15.4-py3-none-any.whl (16 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached platformdirs-4.2.2-py3-none-any.whl (18 kB)
Using cached PyJWT-2.8.0-py3-none-any.whl (22 kB)
Using cached pyOpenSSL-24.1.0-py3-none-any.whl (56 kB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached setuptools-70.3.0-py3-none-any.whl (931 kB)
Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Using cached urllib3-1.26.19-py2.py3-none-any.whl (143 kB)
Using cached zipp-3.19.2-py3-none-any.whl (9.0 kB)
Using cached exceptiongroup-1.2.2-py3-none-any.whl (16 kB)
Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)
Using cached matplotlib-3.9.1-cp39-cp39-manylinux_

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached fosforio-1.0.2-py3-none-any.whl.metadata (13 kB)
  Using cached pandas-2.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached numpy-2.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
Using cached fosforio-1.0.2-py3-none-any.whl (20 kB)
Using cached pandas-2.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
Using cached numpy-2.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.3 MB)
Using cached python_dateutil-2

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Preparing metadata (setup.py) ... done
  Using cached urllib3-1.26.19-py2.py3-none-any.whl.metadata (49 kB)
  Using cached numpy-1.26.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached scipy-1.13.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached charset_normalizer-3.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.7 MB

In [ ]:
!pip install seaborn scipy xgboost pandas dice-ml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn pydantic-settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [3]:
from refractio import snowflake
snowflake.get_connection(connection_name="Final hr attrition data")

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
Exception occurred in getting snowflake connection: 'connectionSources'


In [4]:
df = snowflake.get_dataframe("Final_HR_Attrition_Data")
df

,SALARY,SENIORITY,TENURE_MONTHS,MONTHS_AFTER_COLLEGE,BIRTH_YEAR,AGE,MAPPED_ROLE_CLEAN,SEX,ETHNICITY,HOSPITAL_TYPE,...,OVER_TIME_HOURS,CHURN,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION,JOB_SATISFACTION,MARTIAL_STATUS,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,RELATIONSHIP_SATISFACTION,WORK_LIFE_BALANCE
0,31657.073,1,13,161,1986,38,occupational,F,Black,Psychiatric,...,7,True,Travel_Rarely,High,Medium,Married,18,Excellent,Medium,Better
1,82697.324,2,10,72,1988,36,social,M,API,Acute Care - Department of Defense,...,6,False,Travel_Rarely,High,Very High,Married,19,Excellent,Low,Good
2,67291.312,1,65,7,1986,38,nurse,M,Black,Acute Care - Department of Defense,...,9,False,Non-Travel,High,Low,Married,14,Excellent,Very High,Bad
3,41186.945,1,36,33,1988,36,technologist,F,Hispanic,Psychiatric,...,8,False,Travel_Rarely,Low,Very High,Divorced,21,Low,Medium,Bad
4,54826.128,2,9,23,1990,34,physician,M,Black,Childrens,...,11,True,Travel_Rarely,High,Very High,Single,16,Good,High,Better
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,97520.177,5,45,113,1969,55,physician,M,Hispanic,Acute Care Hospitals,...,1,False,Travel_Rarely,High,High,Divorced,20,Outstanding,Low,Best
299996,55666.168,2,32,5,1987,37,emt,M,White,Childrens,...,9,True,Travel_Rarely,Medium,Low,Married,23,Low,Very High,Best
299997,49866.716,2,34,4,1986,38,physician,F,White,Childrens,...,0,True,Travel_Rarely,Medium,Medium,Single,14,Excellent,Medium,Bad
299998,77706.113,2,6,26,1996,28,physician,F,API,Childrens,...,1,False,Travel_Rarely,High,High,Single,11,Good,Very High,Good


In [5]:
print(df.isnull().sum())

SALARY                           0
SENIORITY                        0
TENURE_MONTHS                    0
MONTHS_AFTER_COLLEGE             0
BIRTH_YEAR                       0
AGE                              0
MAPPED_ROLE_CLEAN                0
SEX                              0
ETHNICITY                        0
HOSPITAL_TYPE                    0
HOSPITAL_OWNERSHIP               0
COMPANY_NAME                     0
CITY                             0
STATE                            0
DISTANCE                         0
DEGREE_CLEAN                     0
SCHOOL_END_DATE                  0
JOB_START_DATE                   0
JOB_END_DATE                 90102
USER_ID                          0
OVER_TIME_HOURS                  0
CHURN                            0
BUSINESS_TRAVEL                  0
ENVIRONMENT_SATISFACTION         0
JOB_SATISFACTION                 0
MARTIAL_STATUS                   0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
RELATIONSHIP_SATISFA

In [6]:
df = df.dropna()

In [7]:
print(df.isnull().sum())

SALARY                       0
SENIORITY                    0
TENURE_MONTHS                0
MONTHS_AFTER_COLLEGE         0
BIRTH_YEAR                   0
AGE                          0
MAPPED_ROLE_CLEAN            0
SEX                          0
ETHNICITY                    0
HOSPITAL_TYPE                0
HOSPITAL_OWNERSHIP           0
COMPANY_NAME                 0
CITY                         0
STATE                        0
DISTANCE                     0
DEGREE_CLEAN                 0
SCHOOL_END_DATE              0
JOB_START_DATE               0
JOB_END_DATE                 0
USER_ID                      0
OVER_TIME_HOURS              0
CHURN                        0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
JOB_SATISFACTION             0
MARTIAL_STATUS               0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
RELATIONSHIP_SATISFACTION    0
WORK_LIFE_BALANCE            0
dtype: int64


In [8]:
df = df.drop(["USER_ID", "JOB_START_DATE", "JOB_END_DATE", "SCHOOL_END_DATE"], axis = 1)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
CATEGORICAL_COLUMNS = ["MAPPED_ROLE_CLEAN","SEX", "ETHNICITY","HOSPITAL_TYPE", "HOSPITAL_OWNERSHIP","COMPANY_NAME","CITY","STATE","DISTANCE", 
                       "DEGREE_CLEAN","BUSINESS_TRAVEL","ENVIRONMENT_SATISFACTION","JOB_SATISFACTION","MARTIAL_STATUS","PERFORMANCE_RATING","RELATIONSHIP_SATISFACTION","WORK_LIFE_BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR","AGE", "OVER_TIME_HOURS", "PERCENTAGE_SALARY_HIKE",]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [11]:
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS 

In [12]:
feature_columns = [col for col in feature_columns if col in df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in df.columns]

In [13]:
X = df[feature_columns]
Y = df[LABEL_COLUMNS]

In [14]:
X.head()

,MAPPED_ROLE_CLEAN,SEX,ETHNICITY,HOSPITAL_TYPE,HOSPITAL_OWNERSHIP,COMPANY_NAME,CITY,STATE,DISTANCE,DEGREE_CLEAN,...,RELATIONSHIP_SATISFACTION,WORK_LIFE_BALANCE,SALARY,SENIORITY,TENURE_MONTHS,MONTHS_AFTER_COLLEGE,BIRTH_YEAR,AGE,OVER_TIME_HOURS,PERCENTAGE_SALARY_HIKE
0,occupational,F,Black,Psychiatric,Government - Local,24-7 Bright Star Healthcare LLC,Jacksonville,FL,5-10 Miles,Bachelors Degree,...,Medium,Better,31657.073,1,13,161,1986,38,7,18
1,social,M,API,Acute Care - Department of Defense,Government - Hospital District or Authority,Mayo Clinic,Deltona,FL,<2 Miles,Masters Degree,...,Low,Good,82697.324,2,10,72,1988,36,6,19
2,nurse,M,Black,Acute Care - Department of Defense,Physician,Cleveland Clinic,Reno,NV,<2 Miles,Masters Degree,...,Very High,Bad,67291.312,1,65,7,1986,38,9,14
3,technologist,F,Hispanic,Psychiatric,Government - State,24-7 Bright Star Healthcare LLC,Phoenix,AZ,5-10 Miles,Undergraduate Diploma,...,Medium,Bad,41186.945,1,36,33,1988,36,8,21
4,physician,M,Black,Childrens,Voluntary non-profit - Other,McKesson Corp.,Janesville,WI,5-10 Miles,Doctoral Degree,...,High,Better,54826.128,2,9,23,1990,34,11,16


In [15]:
Y.head()

,CHURN
0,True
1,False
2,False
3,False
4,True


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.20, random_state = 42)

In [17]:
## train_df and test_df are both snowpark dataframes
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
)

numerical_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(clip=True)
)

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS),
        ('num', numerical_transformer, NUMERICAL_COLUMNS)
    ]
)


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

pipeline.fit(X_train, y_train)

result = pipeline.predict(X_test)

In [18]:
result

array([0, 1, 1, ..., 1, 1, 1])

In [19]:
from joblib import dump, load
filename = "HR_Attrition_ml_model.joblib"
dump(pipeline, filename)

['HR_Attrition_ml_model.joblib']

In [20]:
import requests

In [21]:
y_train.columns

Index(['CHURN'], dtype='object')

In [22]:
model = load(filename)

In [23]:
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)

In [25]:
from refractml import *

from refractml.constants import MLModelFlavours

ModuleNotFoundError: No module named 'refractml'

In [24]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data = pd.DataFrame(payload_dict,index=[0])
    prediction = str(model.predict(data)[0])
    return prediction

NameError: name 'scoring_func' is not defined

In [ ]:
import requests
payload = str(X_test.iloc[123].to_dict())
req = requests.Request()
req.json = {"payload": payload}

print(score(pipeline, req))

In [ ]:
## registering the model in refract.
tmp = register_model(pipeline, 
               score, 
               name="HR_Analytic_ml_model_Final", 
               description="Final_HR_ANALYTICS_model_trained_using _ml",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install seaborn \\n pip install snowflake-connector-python[pandas] \\n pip install joblib==1.3.2 scikit-learn=1.3.2",
               y_true=y_test,
               y_pred=y_pred, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True,
               prob=y_prob,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['NOT LEFT','LEFT'],
               kyd=True, kyd_score = True)


In [ ]:
payload  = {"payload": X_test.iloc[0].to_dict()}
payload